In [39]:
from argparse import Namespace
from pathlib import Path
from glob import glob
import json

scenes = []
for path in glob("../output/*"):
    try:
        namespace = eval((Path(path) / "cfg_args").read_text())
        results = None
        if (Path(path) / "results.json").exists():
            results = json.loads((Path(path) / "results.json").read_text())
            scenes.append((namespace,results))
    except FileNotFoundError as e:
        print(e)

[Errno 2] No such file or directory: '../output/point_cloud/cfg_args'
[Errno 2] No such file or directory: '../output/livingroom_slow_depth/cfg_args'
[Errno 2] No such file or directory: '../output/livingroom_slow/cfg_args'


In [40]:
import pandas as pd

results_df = []
for namespace, results in scenes:
    if namespace.source_path.split("/")[-1]=="chair_background2":
        for name,res in results.items():
            if "per" not in name:
                r = {
                    "train_run": namespace.model_path.split("/")[-1],
                    "train_mode": "depth" if "depth" in namespace.model_path else "normal",
                    "render_mode": "texture" if "texture" in name else "normal",
                    "method_name":"_".join(name.split("_")[:-1]),
                    "iteration":int(name.split("_")[-1]),
                    **res
                }
                if r["iteration"] in [2000, 3000, 4000, 5000, 7000, 10000, 13000, 16000, 20000, 25000, 30000, 35000, 40000, 47000, 55000, 70000]:
                    results_df.append(r)


results_df = pd.DataFrame(results_df)

In [41]:
results_df = results_df.sort_values("num_gaussians")
results_df["run"] = results_df["train_mode"] + results_df["render_mode"]
is_texture = (results_df["render_mode"]=="texture")
results_df["memory_needed"] = results_df["num_gaussians"]*(48*(~is_texture)+11)*4 + 300000*50*is_texture

In [42]:
results_df

,train_run,train_mode,render_mode,method_name,iteration,SSIM,PSNR,LPIPS,num_gaussians,run,memory_needed
8,chair_background2_slow_depth,depth,texture,texture_alpha,2000,0.727880,21.774786,0.220484,3000,depthtexture,15132000
19,chair_background2_slow,normal,normal,normal_alpha,2000,0.328636,20.897512,0.677142,3118,normalnormal,735848
30,chair_background2_slow,normal,texture,texture_alpha,2000,0.432295,20.858892,0.376621,3118,normaltexture,15137192
21,chair_background2_slow,normal,normal,normal_alpha,3000,0.358572,21.515293,0.630679,3514,normalnormal,829304
28,chair_background2_slow,normal,texture,texture_alpha,3000,0.460262,21.445301,0.352963,3514,normaltexture,15154616
17,chair_background2_slow,normal,normal,normal_alpha,4000,0.398918,22.002769,0.586894,4457,normalnormal,1051852
22,chair_background2_slow,normal,texture,texture_alpha,4000,0.490352,21.894995,0.331932,4457,normaltexture,15196108
26,chair_background2_slow,normal,texture,texture_alpha,5000,0.548741,22.602959,0.296196,5305,normaltexture,15233420
14,chair_background2_slow,normal,normal,normal_alpha,5000,0.433030,22.407787,0.558102,5305,normalnormal,1251980
0,chair_background2_slow_depth,depth,texture,texture_alpha,4000,0.769883,23.218552,0.190198,5880,depthtexture,15258720


In [43]:
import plotly.express as px

def pareto_line(df,x,y,color,fn="max"):
    df = df.copy()
    if fn=="max":
        df = df[df[y]==df.groupby(color)[y].cummax()]
    else:
        df = df[df[y]==df.groupby(color)[y].cummin()]
    
    df = df[df["num_gaussians"] != df.groupby(color)["num_gaussians"].shift(-1)]
    
    return px.line(df, x=x, y=y, color=color, log_x=True)

df = results_df[(results_df["run"]=="normalnormal") | (results_df["run"]=="depthtexture") | (results_df["run"]=="normaltexture")]
fig = pareto_line(df,"num_gaussians","PSNR","run")
fig.show()
fig = pareto_line(df,"num_gaussians","LPIPS","run",fn="min")
fig.show()
fig = pareto_line(df,"num_gaussians","SSIM","run")
fig.show()

In [12]:
pareto_line(df,"memory_needed","PSNR","run").show()
pareto_line(df,"memory_needed","LPIPS","run",fn="min").show()
pareto_line(df,"memory_needed","SSIM","run").show()

In [38]:
best_results = results_df.loc[results_df.groupby("run")["PSNR"].idxmax()]
formatted_table = best_results[["PSNR","SSIM","LPIPS"]].applymap(lambda x: f"{x:.3f}" if x<1 else f"{x:.2f}")
formatted_table["#Primitives"] = best_results["num_gaussians"].apply(lambda x: f"{x//1000}k")
formatted_table["Memory"] = best_results["memory_needed"].apply(lambda x: f"{x//1000000} MB")

formatted_table["Depth Supervision"] = best_results["train_mode"].apply(lambda x: {"depth": "Yes", "normal":"No"}[x])
formatted_table["Textured Rendering"] = best_results["render_mode"].apply(lambda x: {"texture": "Yes", "normal":"No"}[x])
print(formatted_table.loc[:,["Textured Rendering","Depth Supervision","PSNR","SSIM","LPIPS","#Primitives","Memory"]].to_latex())

\begin{tabular}{llllllll}
\toprule
 & Textured Rendering & Depth Supervision & PSNR & SSIM & LPIPS & #Primitives & Memory \\
\midrule
10 & Yes & Yes & 30.21 & 0.909 & 0.070 & 44k & 16 MB \\
18 & No & No & 32.03 & 0.921 & 0.100 & 502k & 118 MB \\
\bottomrule
\end{tabular}



In [34]:
px.line(x)

: 